In [ ]:
import pandas as pd

df = pd.read_excel('../seoul_list(1).xlsx')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df_drop = df.drop(columns=['충전기 ID','지역','제한'])

In [ ]:
df_sort_gu = df_drop.sort_values(by=['시군구','Unnamed: 13'], ascending=True)
df_sungjung_gu = df_sort_gu[df_sort_gu['시군구']>='성동구']
df_sungjung_gu.to_excel('test1.xlsx')

In [ ]:
test = pd.read_excel('./test.xlsx')
# test : 동 찾으려고 만든 엑셀파일. 동 없는 애들이 맨아래있었음.
test.tail()

In [ ]:
import re
# 정규표현식으로 동만 남기기
test[3959:]
# 인덱스 3959부터 찾은 동들

In [ ]:
test.loc[3959:,"행정동"]

In [ ]:
def cleaning(x):
    if str(x).find('|') != -1:
        return str(x).split("|")[0].split()[-1]
    return str(x)

test.loc[3959:,'행정동'] = test.loc[3959:,'행정동'].apply(cleaning).apply(lambda x:x.replace(' ',''))

In [ ]:
test.loc[3959:,'행정동']

In [ ]:
test.to_excel('test_comp.xlsx')

In [ ]:
sungjung = pd.read_excel('test_comp.xlsx')
sungjung.head()

In [ ]:
sungjung.drop(columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'])

In [ ]:
def clean_type(x):
    if str(x).find("완속") != -1:
        return "고속"
    return "완속"

In [ ]:
sungjung['타입'] = sungjung['타입'].apply(clean_type).apply(lambda x:x.replace(' ',''))
sungjung.head()

In [ ]:
sungjung.drop(columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'])

In [ ]:
sungjung.to_excel('typeClear.xlsx')

In [ ]:
df1 = pd.read_excel('typeClear.xlsx')
df1.drop(columns=['Unnamed: 0','Unnamed: 0.1', 'Unnamed: 0.1.1','Unnamed: 0.1.1.1'])

In [ ]:
groups = df1.groupby('행정동')
groups.size()

In [ ]:
# 제한x : 0, 제한 : 1
def clean_bigo(x):
    if str(x).find("제한") != -1:
        return 1
    return 0

# 24시간 : 1, 시간제한 : 0
def clean_runtime(x):
    if str(x).find("시간") != -1:
        return 1
    return 0

In [ ]:
df1['비고'] = df1['비고'].apply(clean_bigo)
df1['이용가능시간'] = df1['이용가능시간'].apply(clean_runtime)

In [ ]:
df1.head()
df1.drop(columns=['Unnamed: 0','Unnamed: 0.1', 'Unnamed: 0.1.1','Unnamed: 0.1.1.1'])

In [ ]:
def classifyAll(row):
    lst = [0]*9
    lst[0] = row[-1]
    # 24시간이면
    if row[-4] == 1:
        # 고속이면
        if row[-7] == '고속':
            lst[1]+=1
            if row[-2] == 1:
                lst[2]+=1
        else:
            lst[5]+=1
            if row[-2] == 1:
                lst[6]+=1
    else:
        if row[-7] == '고속':
            lst[3]+=1
            if row[-2] == 1:
                lst[4]+=1
        else:
            lst[7]+=1
            if row[-2] == 1:
                lst[8]+=1
    return lst

In [ ]:
df2 = df1

In [ ]:
df2.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1','Unnamed: 0.1.1.1'])

In [ ]:
fin = df2.apply(classifyAll, axis=1)

In [ ]:
type(fin)

In [ ]:
col_name = ['dong','fulltime_fch','fulltime_fch_constraint','nonfulltime_fch','nonfulltime_fch_constraint','fulltime_sch','fulltime_sch_constraint','nonfulltime_sch','nonfulltime_sch_constraint']

In [ ]:
fin_df = pd.DataFrame(fin.values.tolist())
fin_df.columns=col_name
fin_df

In [ ]:
final = fin_df.groupby('dong').sum()

In [ ]:
final.to_excel('final_gu.xlsx')